In [1]:
import os

import numpy as np
import pandas as pd

import cv2
import zipfile
from pdf2image import convert_from_path

In [2]:
from openai import OpenAI
from pinecone import Pinecone
from dotenv import load_dotenv

In [3]:
cur_dir = os.getcwd()
zip_file_path = 'Electricity_bills.zip'
data_dir = os.path.join(cur_dir,'data\\')
pdf_dir = os.path.join(data_dir, 'pdf\\')

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)

with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    zip_file.extractall(pdf_dir)

In [4]:
jpeg_dir = os.path.join(data_dir, 'jpeg\\')
if not os.path.exists(jpeg_dir):
    os.mkdir(jpeg_dir)


for pdf in os.listdir(pdf_dir):
    image = convert_from_path(os.path.join(pdf_dir,pdf))
    image[0].save(os.path.join(jpeg_dir,pdf[:-4]+'.jpeg'), 'JPEG')

In [5]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [6]:
def vision_embed_file(file_name, multi_modal_model='gpt-4.1-mini', embedding_model='text-embedding-3-small'):
  def create_file(file_path):
    with open(file_path, "rb") as file_content:
      result = client.files.create(
          file=file_content,
          purpose="vision",
      )
      return result.id

  file_id = create_file(file_name)

  response = client.responses.create(
      model = multi_modal_model,
      input=[{
            'role':'user',
            'content':[{
                'type': 'input_text',
                    'text': 'what\'s in this image?'},
                {'type':'input_image',
                    'file_id':file_id}
            ]
      }]
  )
  caption = response.output_text
  embedding_object = client.embeddings.create(input=caption, model=embedding_model)
  vector = embedding_object.data[0].embedding

  # Display Results
  # Display Image from URL
  # os.system('wget %s' %image_url_)
  # file_name=str(image_url_).split("/")[-1]
#   print(file_name)
#   img = cv2.imread(file_name)
#   cv2_imshow(img)
#   print(caption)
#   print("Summary Length in characters:"+str(len(caption)))


  return_dict = {'image_caption': caption, 'file_id':file_id, 'embedding':vector}

  return return_dict



In [16]:
def get_embeddings(text, model="text-embedding-3-small"):
    text = text.replace("\n"," ")
    return client.embeddings.create(input=text, model=model).data[0].embedding

In [7]:
df = pd.DataFrame(columns=['id', 'values', 'metadata'])

i=0
for jpeg in os.listdir(jpeg_dir):
    file_path = os.path.join(jpeg_dir, jpeg)
    embedding = vision_embed_file(file_path)
    df.loc[i] = [embedding['file_id'], embedding['embedding'], {'caption':embedding['image_caption']}]
    i += 1

df.to_csv(os.path.join(data_dir, 'embeds.csv'), index=False)

In [8]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index('retrieval-augmented-generation')
index.describe_index_stats()


c:\Users\Rami Alrwais\Desktop\Project3\envrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 23}},
 'total_vector_count': 23,
 'vector_type': 'dense'}

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [ ]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [9]:
def prepare_DF(df):
  import json,ast
  try: df=df.drop('Unnamed: 0',axis=1)
  except: print('Unnamed Not Found')
  df['values']=df['values'].apply(lambda x: np.array([float(i) for i in x.replace("[",'').replace("]",'').split(',')]))
  df['metadata']=df['metadata'].apply(lambda x: ast.literal_eval(x))
  return df

In [10]:
index_df = prepare_DF(pd.read_csv(os.path.join(data_dir,'embeds.csv')))

Unnamed Not Found


In [11]:
upsert_vectors = list(index_df.itertuples(index=False, name=None))

In [12]:
index.upsert(vectors=upsert_vectors)

{'upserted_count': 20}

In [13]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 23}},
 'total_vector_count': 23,
 'vector_type': 'dense'}

In [21]:
sample_text = "What bills are due?"

In [22]:
test_embeddings = get_embeddings(sample_text)

In [18]:
index.query(vector=test_embeddings, top_k=5,include_metadata=True)

{'matches': [{'id': 'file-4DbDBqnxMH7WnmQZaX2y9w',
              'metadata': {'caption': 'This image is of an electricity bill '
                                      'from AEP Ohio. Here are the key details '
                                      'from the bill:\n'
                                      '\n'
                                      '- Billing period: 12/07/2020 to '
                                      '01/02/2021 (26 days)\n'
                                      '- Account number: 893-180137-3-3\n'
                                      '- Meter number: 268714261\n'
                                      '- Service address: Elizabeth Hernandez, '
                                      '1336 River Rd, Marietta, OH 44503\n'
                                      '- Amount due: $218.05 (due on or before '
                                      'January 13, 2021)\n'
                                      '\n'
                                      'Line item charges include:\n'
 

In [23]:
def get_context(query, embed_model = 'text-embedding-3-small',k=5,index=index):
    query_embeddings = get_embeddings(query,model=embed_model)
    pinecone_response = index.query(vector=query_embeddings,top_k=k,include_metadata=True)
    contexts = [item['metadata']['caption'] for item in pinecone_response['matches']]
    return contexts, query

In [24]:
get_context("Tell me about Avg. monthly usage")

(['This image shows a utility bill from AEP Ohio. \n\nKey details include:\n\n- Billing period: 12/07/2020 - 01/02/2021 (26 days)\n- Account number: 893-180137-3-3\n- Meter number: 268714261\n- Service address: Elizabeth Hernandez, 1336 River Rd, Marietta, OH 44503\n- Amount due: $218.05 (due on or before January 13, 2021)\n\nLine item charges breakdown:\n- Supply Charge (1151 kWh): $102.62\n- Delivery Charge: $70.37\n- Customer Charge: $14.62\n- Retail Stability / Riders: $0.52\n- Estimated Taxes & Assessments: $13.71\n- Total Current Charges: $201.84\n\nUsage details:\n- Total usage for the past 12 months: 9,888 kWh\n- Average monthly usage: 824 kWh\n- Average daily cost: $7.48\n- Average temperature: 32 °F\n- Meter read details: previous reading 2,093,749; current reading 2,094,900; usage 1,151 kWh\n\nA payment stub is included at the bottom with the same amount due and due date.',
  'This image is a utility bill from AEP Ohio, detailing the electricity usage and charges for a billi

In [25]:
def augmented_query(user_query, embed_model='text-embedding-3-small',k=5):
    contexts,query = get_context(user_query,embed_model = embed_model,k=k)
    return "\n\n--------------------------\n\n".join(contexts)+"\n\n--------------------------\n\n" + query

In [26]:
my_question = "Tell me about useage?"
augg = augmented_query(my_question)
print(augg)

This image is a utility bill from AEP Ohio, detailing the electricity usage and charges for a billing period from February 1, 2022, to March 3, 2022 (30 days). It includes the following information:

- Account number: 315-821911-1-3
- Meter number: 915230394
- Service address: Linda Wilson, 1271 Main St, Canton, OH 44301
- Total amount due: $114.73, payable on or before March 11, 2022

Line Item Charges:
- Supply Charge (586 kWh): $51.75
- Delivery Charge: $44.52
- Customer Charge: $7.64
- Retail Stability / Riders: $5.26
- Estimated Taxes & Assessments: $5.56
- Total Current Charges: $114.73

Usage Details:
- Total usage for the past 12 months: 5229 kWh
- Average monthly usage: 435 kWh
- Average daily cost: $3.70
- Average temperature: 40°F
- Meter read details: Previous reading 9263658, Current reading 9264244, Usage 586 kWh

There is also a payment stub at the bottom with the account number, amount due, and due date for return with payment. The bill provides contact information for 

In [27]:
def ask_gpt_response(system_prompt, user_prompt, model='gpt-5-chat-latest'):
  response = client.responses.create(
      model=model,
      input=[
          {"role":"developer",
          "content":system_prompt},
          {"role":"user",
           "content":user_prompt}])
  return response.output_text, response

In [28]:
import textwrap

In [29]:
primer = f"""
You are a knowledgeable assistant specialized in answering questions about electric utility bills. 
You provide accurate and clear explanations based solely on the bill details and information provided above each question. 
If the information is not sufficient to answer the question, respond truthfully with, "I don't know."
"""

In [30]:
print(ask_gpt_response(system_prompt=primer, user_prompt=augmented_query('Give me some analysis of recent bills?')))

('Here’s an analysis of the **recent AEP Ohio electric bills** you described:  \n\n---\n\n### 🔹 Summary of the Four Bills\n\n| Customer | Billing Period | Days | Usage (kWh) | Amount Due | Avg. Daily Cost |\n|-----------|----------------|------|--------------|-------------|----------------|\n| **Susan Moore (Columbus)** | 03/08/2018 – 04/03/2018 | 26 | 500 | **$96.91** | $3.59 |\n| **Charles Taylor (Cincinnati)** | 09/16/2018 – 10/21/2018 | 35 | 292 | **$77.45** | $2.15 |\n| **Mary Miller (Dayton)** | 10/21/2019 – 11/20/2019 | 30 | 1439 | **$195.30** | $6.30 |\n| **Elizabeth Hernandez (Marietta)** | 12/07/2020 – 01/02/2021 | 26 | 1151 | **$218.05** | $7.48 |\n\n---\n\n### 🔹 Observations & Trends\n\n1. **Electric Usage Variation**\n   - Usage ranges widely from **292 kWh** to **1439 kWh**, showing how consumption can differ based on household size, season, or heating type.\n   - The lowest usage (292 kWh) was in a **mild fall month**, while higher usage (1151–1439 kWh) occurred in **col